This notebook provides the necessary ingredients for our TRF method (Algorithm 1 in the paper)

In [2]:
from scipy.optimize import least_squares
import math
import scipy.special,scipy.linalg
import numpy as np
import time
from matplotlib import pyplot as plt
pi = np.pi

In [3]:
### The function below takes as input the normalized data and returns the parameter tau used in our TRF alforithm
def estim_tau(X):
    tau = np.mean(np.diag(X.T@X))
    
    return tau


### The function below takes as input the parameter tau (computed from data) using the function estim_tau(X), and return
### the thresholds s_minus and s_plus used to set the ternary function sigma_ter
def compute_thresholds(tau):
    F = lambda x: ((np.exp(-x[0] ** 2 / tau) + np.exp(-x[1] ** 2 / tau)) / np.sqrt(2 * pi * tau) - np.exp(-tau / 2),
                   (-x[0] * np.exp(-x[0] ** 2 / tau) + x[1] * np.exp(-x[1] ** 2 / tau)) / (
                       np.sqrt(2 * pi * tau ** 3)) - np.exp(-tau / 2) / 2)
    ### relu
    #F = lambda x: ((np.exp(-x[0] ** 2 / tau) + np.exp(-x[1] ** 2 / tau)) / np.sqrt(2 * pi * tau) - 1/2,
    #               (-x[0] * np.exp(-x[0] ** 2 / tau) + x[1] * np.exp(-x[1] ** 2 / tau)) / (
    #                   np.sqrt(2 * pi * tau ** 3)) - 1/np.sqrt(8*pi*tau))

    res = least_squares(F, (1, 1), bounds=((0, 0), (1, 1)))
    s_minus = -min(res.x)
    s_plus = max(res.x)
    return s_minus , s_plus 


### Given a sparsity level eps, data size n, data dimension p, the function below return an i.i.d zero mean unit variance 
### random matrix taking values -1, 0, 1
def ternary_weight(eps, n, p):
    elements = [-1, 0, 1]
    probabilities = [(1-eps)/2, eps, (1-eps)/2]
    W = np.random.choice(elements, (n,p), p=probabilities)
    W = scipy.sparse.csr_matrix(W)
    return W


### Given s_minus, s_plus, and projected data Z,  the function below returns the activation \sigma^{ter}(Z)
def gen_sig(Z, s1=None, s2=None):
    sig =  (Z>(np.sqrt(2)*s_plus)).astype(int) - (Z<(np.sqrt(2)*s_minus)).astype(int)
    return sig